In [1]:
import pandas as pd
import json
import numpy as np
import tqdm as tqdm
import ast
import re

### OpenAI API playground

In [15]:
from openai import OpenAI
import os

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],
)

hs = 'All muslims are terrorists'
cs1 = 'Not all muslims are terrorists'
cs2 = 'Jews are a bunch of thieves'
cs3 = 'Islamphobia is not a prevelant scheme in Europe'

system_description = '''
You will be given one counterspeech (also called as counter-argument or counter-narrative) written for a hateful statement (hate speech). Your task is to rate the counterspeech on one metric. Please make sure you read and understand these instructions carefully. 

Evaluation Criteria:
Relevance (1-5) - This dimension evaluates whether the counterspeech addresses the central theme or subject of the hate speech. 

Evaluation Steps:
1. **Read and Understand the Hate Speech**: Begin by reading the hate speech thoroughly. Understand the central theme, subject, or the specific point of hate it is addressing. This might involve recognizing the target group, the type of hateful message (e.g., stereotypes, derogatory remarks), and the context in which it is presented.

2. **Evaluate Relevance**:
   - **Score 5 (Highly Relevant)**: The counterspeech directly addresses the central theme or subject of the hate speech.
   - **Score 4 (Mostly Relevant)**: The counterspeech mainly addresses the central theme of the hate speech but may include minor elements or arguments that are not directly related.
   - **Score 3 (Moderately Relevant)**: The counterspeech addresses the hate speech but also includes significant unrelated content or deviates from the main point at times.
   - **Score 2 (Slightly Relevant)**: The counterspeech only partially addresses the hate speech, with a considerable portion of the content being unrelated or off-topic.
   - **Score 1 (Not Relevant)**: The counterspeech fails to address the central theme or subject of the hate speech. It is largely or entirely off-topic.

3. **Consider Contextual Factors**: Sometimes, the relevance of a counterspeech can be influenced by contextual factors such as cultural references, historical events, or social nuances. Ensure these factors are considered in your evaluation, especially if the hate speech or counterspeech involves specific references or subtleties.

4. **Finalize Your Score**: After careful consideration, assign a score from 1 to 5 (Likert Scale) based on the relevance criteria. 

Ensure that the response is STRICTLY in JSON format.
'''.strip()


response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  response_format={ "type": "json_object" },
  messages=[
        {"role": "system", "content": system_description},
        {"role": "user", "content": f"Hate Speech 1: '{hs}'\nCounterspeech 1: '{cs1}'"},
        {"role": "user", "content": f"Hate Speech 2: '{hs}'\nCounterspeech 2: '{cs2}'"},
        {"role": "user", "content": f"Hate Speech 3: '{hs}'\nCounterspeech 3: '{cs3}'"},
    ]
)

KeyError: 'OPENAI_API_KEY'

In [14]:
from pprint import pprint
pprint(response)

ChatCompletion(id='chatcmpl-8pVz1CvKg8Nknrlk3kO7X6B1rc8aH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "Relevance": 2\n}', role='assistant', function_call=None, tool_calls=None))], created=1707288363, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_69829325d0', usage=CompletionUsage(completion_tokens=10, prompt_tokens=541, total_tokens=551))


In [13]:
for x in response.choices:
    print(x.message.content)

{
  "Relevance": 2
}


In [3]:
# import gspread
# import pandas as pd
# from oauth2client.service_account import ServiceAccountCredentials

# # Set up the scope and credentials
# scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('/home/ameyh/cred/google.json', scope)
# client = gspread.authorize(credentials)

# # Open the Google Sheet by its title
# sheet = client.open('Counter Speech Project - Dashboard')

# # Locate a subsheet by its name
# subsheet = sheet.worksheet('untagged-dataset')

# # Read the subsheet into a Pandas DataFrame
# df = pd.DataFrame(subsheet.get_all_records())

df = pd.read_csv('/home/ameyh/counterspeech-EVAL/preds/results_combined.csv')

print(df.shape)
df.head(2)

(19555, 25)


,hatespeech,counterspeech,csType,source,prompt_relevance_score,prompt_aggressiveness_score,prompt_coherence_score,system_description,prediction_(prompt_aggressiveness_score)_(gpt-4),prediction_(prompt_coherence_score)_(gpt-3.5-turbo),...,"pc_score_(hs,cs)","cd_score_(hs,cs)","aq_score_(hs,cs)","bert_score_(hs,cs)",toxicity_score_(cs),obscene_score(cs),identity_attack_score(cs),insult_score(cs),"bm25_(hs,cs)","mauve_(hs,cs)"
0,Maybe the UN could talk to those asian and afr...,The US is the second most polluting country in...,Informative,Human,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Aggressiveness': '1'\n},{ \n'Coherence': '4'\n},...,0.990696,0.328718,0.598695,0.862366,0.000784,0.000033,0.000128,0.000171,-0.885247,namespace(divergence_curve=array([[1.00000000e...
1,Maybe the UN could talk to those asian and afr...,Doesn't everyone on the planet have a responsi...,Questioning,Human,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Aggressiveness': '1'\n},{ \n'Coherence': '4'\n},...,0.995955,0.115010,0.601427,0.855926,0.000602,0.000022,0.000120,0.000142,-0.144827,namespace(divergence_curve=array([[1.00000000e...


In [4]:
df.source.value_counts()

Human       13823
dialoGPT     2914
QUARC        1900
GPS           918
Name: source, dtype: int64

In [14]:
# df = df[df.source == 'Human']
# print(df.shape)

In [5]:
hatespeeches = df.hatespeech.values.tolist()
counterspeeches = df['counterspeech'].values.tolist()

print(hatespeeches[0], counterspeeches[0])

Maybe the UN could talk to those asian and african nations responsible for 90%+ of the pollution in the oceans' instead of insisting on this bullshit about climate change. The US is the second most polluting country in the world - as the world’s biggest industrial and commercial power. We are all to blame here and need to work hand in hand to create sustainable change, as opposed to pointing the finger to others.


In [6]:
from prompts import relevance_score_prompt, coherence_score_prompt, aggressiveness_score_prompt, suitableness_score_prompt


hs = 'Muslims are terrorists'
cs = 'Not all Muslims are terrorists'

print(relevance_score_prompt(hs, cs))


You will be given one counterspeech (also called as counter-argument or counter-narrative) written for a hateful statement (hate speech). 
Your task is to rate the counterspeech on one metric. 
Please make sure you read and understand these instructions carefully. 

Evaluation Criteria:
Relevance (1-5) - This dimension evaluates whether the counterspeech addresses the central theme or subject of the hate speech. 

Evaluation Steps:
1. **Read and Understand the Hate Speech**: Begin by reading the hate speech thoroughly. Understand the central theme, subject, or the specific point of hate it is addressing. This might involve recognizing the target group, the type of hateful message (e.g., stereotypes, derogatory remarks), and the context in which it is presented.

2. **Evaluate Relevance**:
   - **Score 5 (Highly Relevant)**: The counterspeech directly addresses the central theme or subject of the hate speech.
   - **Score 4 (Mostly Relevant)**: The counterspeech mainly addresses the ce

In [18]:
df['prompt_relevance_score'] = df.apply(lambda row: relevance_score_prompt(row['hatespeech'], row['counterspeech']), axis=1)
sample = df.iloc[0]['prompt_relevance_score']
print(sample)


You will be given one counterspeech (also called as counter-argument or counter-narrative) written for a hateful statement (hate speech). 
Your task is to rate the counterspeech on one metric. 
Please make sure you read and understand these instructions carefully. 

Evaluation Criteria:
Relevance (1-5) - This dimension evaluates whether the counterspeech addresses the central theme or subject of the hate speech. 

Evaluation Steps:
1. **Read and Understand the Hate Speech**: Begin by reading the hate speech thoroughly. Understand the central theme, subject, or the specific point of hate it is addressing. This might involve recognizing the target group, the type of hateful message (e.g., stereotypes, derogatory remarks), and the context in which it is presented.

2. **Evaluate Relevance**:
   - **Score 5 (Highly Relevant)**: The counterspeech directly addresses the central theme or subject of the hate speech.
   - **Score 4 (Mostly Relevant)**: The counterspeech mainly addresses the ce

In [19]:
df['prompt_aggressiveness_score'] = df.apply(lambda row: aggressiveness_score_prompt(row['hatespeech'], row['counterspeech']), axis=1)
sample = df.iloc[0]['prompt_aggressiveness_score']
print(sample)


You will be given one counterspeech (also called as counter-argument or counter-narrative) written for a hateful statement (hate speech). 
Your task is to rate the counterspeech on one metric. 

Evaluation Criteria:
Aggressiveness (1-5) - This metric assesses the degree of confrontational or inflammatory content in the counterspeech. It considers factors such as abusive language, intensity of disagreement, tone, engagement style, and the presence of personal attacks, with higher scores indicating greater aggression.

Evaluation Steps:
1. **Read the Counterspeech Carefully**: Begin by thoroughly reading the counterspeech. Understand the context and the message it intends to convey in response to the hate speech.

2. **Identify Key Elements of Aggression**: Look for elements that contribute to an aggressive tone. This includes:
   - **Abusive Language**: Check for the presence of swear words, slurs, or derogatory terms.
   - **Intensity of Disagreement**: Assess the level of opposition 

In [20]:
df['prompt_coherence_score'] = df.apply(lambda row: coherence_score_prompt(row['hatespeech'], row['counterspeech']), axis=1)
sample = df.iloc[0]['prompt_coherence_score']
print(sample)


You will be given one counterspeech (also called as counter-argument or counter-narrative) written for a hateful statement (hate speech). 
Your task is to rate the counterspeech on one metric. 

Evaluation Criteria:
Coherence (1-5) - This metric assesses how logically and smoothly the ideas or arguments within the counterspeech connect and flow. A coherent counterspeech will present its arguments in an organized manner, making it easy for the reader to follow and understand the counter-narrative being presented.

Evaluation Steps:
1. **Read the Counterspeech**: Carefully go through the counterspeech. Pay attention to how the ideas or arguments are structured and whether the transitions between them are smooth and logical.
2. **Identify Key Aspects of Coherence**:
    - **Logical Flow**: Assess whether the counterspeech presents its arguments or points in a logical sequence that builds upon each previous point.
    - **Clarity of Ideas**: Determine if the ideas are clearly expressed an

In [7]:
df['prompt_suitableness_score'] = df.apply(lambda row: suitableness_score_prompt(row['hatespeech'], row['counterspeech']), axis=1)
sample = df.iloc[0]['prompt_suitableness_score']
print(sample)


You will be given one counterspeech (also called as counter-argument or counter-narrative) written for a hateful statement (hate speech). 
Your task is to rate the counterspeech on one metric. 

Evaluation Criteria:
Choose or not  (1-3) -  This metric measures the likelihood of an annotator choosing a given counterspeech for direct use (without editing) in a real scenario against online hate speech. This assessment considers the counterspeech's suitability, appropriateness, and potential impact on a reader in a real-world context.

Evaluation Steps:
1. **Read the Counterspeech**: Carefully review the counterspeech provided. Remember, this evaluation is about determining the practical applicability of the counterspeech in real-life situations. It's crucial to consider the counterspeech's overall message, tone, and how it addresses the hate speech to decide on its direct usability without modifications.
2. **Assess for Direct Use in Real Scenarios**:
   - **Score 1 (Unlikely to Choose)*

In [21]:
df.to_csv('/home/ameyh/counterspeech-EVAL/preds/results_combined.csv', index=False)

### Post-process LLM predictions

In [6]:
results_df = pd.read_csv('/home/ameyh/counterspeech-EVAL/preds/results_combined.csv')
print(results_df.shape)
results_df.head(1)
# results_df['prediction_gpt-4']

(19556, 11)


,hatespeech,counterspeech,csType,source,prompt_relevance_score,prompt_aggressiveness_score,prompt_coherence_score,system_description,prediction_(prompt_aggressiveness_score)_(gpt-4),prediction_(prompt_coherence_score)_(gpt-3.5-turbo),prediction_(prompt_relevance_score)_(gpt-4)
0,Maybe the UN could talk to those asian and afr...,The US is the second most polluting country in...,Informative,Human,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Aggressiveness': '1'\n},{ \n'Coherence': '4'\n},NaN


In [3]:
# pred_col = 'prediction_(prompt_aggressiveness_score)_(gpt-4)'
# key = 'Aggressiveness'

# results_df[results_df[pred_col].isna()].shape

In [4]:
# sample = results_df.iloc[4]['prediction_gpt-4']
# sample = ast.literal_eval(sample)
# sample['Relevance']

def extract_score(input_string: str, key='Relevance') -> float:
    if not isinstance(input_string, str):
        print(
            f"Invalid input: {input_string}"
        )
        return -1
    """
    try:
        input = ast.literal_eval(input)
        score = input[key]
        score = score.strip().replace(' ','')
        score = float(score)
        return score
    except:
        return -1
    """
     # Use regex to find the number in the string
    match = re.search(r'\d+', input_string)
    
    # If a number is found, convert it to float and return it
    if match:
        return float(match.group())
    else:
        return -1
    
    
# results_df['score_cleaned'] = results_df[pred_col].apply(lambda x: extract_score(x, key=key))
# results_df['score_cleaned'].value_counts()


In [5]:
# results_df['source_and_score'] = results_df.apply(lambda row: (row['source'], row['score_cleaned']), axis=1)
# results_df['source_and_score'].value_counts()

In [17]:
results_df[results_df['score_cleaned'] == -1.0][pred_col].values.tolist()

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'Hate Speech: "Gays a

In [9]:
results_df['relevance_score'] = results_df['prediction_(prompt_relevance_score)_(gpt-4)'].apply(lambda x: extract_score(x, key='Relevance'))
results_df['aggressiveness_score'] = results_df['prediction_(prompt_aggressiveness_score)_(gpt-4)'].apply(lambda x: extract_score(x, key='Aggressiveness'))
results_df['coherence_score'] = results_df['prediction_(prompt_coherence_score)_(gpt-3.5-turbo)'].apply(lambda x: extract_score(x, key='Coherence'))

Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid input: nan
Invalid inpu

In [13]:
results_df['coherence_score'].value_counts()

 4.0    13459
 3.0     4474
 2.0      854
 5.0      557
 1.0      210
-1.0        2
Name: coherence_score, dtype: int64

In [15]:
results_df.head(1)

,hatespeech,counterspeech,csType,source,prompt_relevance_score,prompt_aggressiveness_score,prompt_coherence_score,system_description,prediction_(prompt_aggressiveness_score)_(gpt-4),prediction_(prompt_coherence_score)_(gpt-3.5-turbo),prediction_(prompt_relevance_score)_(gpt-4),relevance_score,aggressiveness_score,coherence_score
0,Maybe the UN could talk to those asian and afr...,The US is the second most polluting country in...,Informative,Human,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Aggressiveness': '1'\n},{ \n'Coherence': '4'\n},NaN,-1.0,1.0,4.0


In [16]:
results_df.to_csv('/home/ameyh/counterspeech-EVAL/preds/results_combined.csv', index=False)

### Rectifier Function and PPI

In [1]:
import numpy as np

# Labeled dataset
labeled_dataset = np.array([4, 3, 2, 5, 4, 3, 5, 2, 1, 4])

# Unlabeled dataset
unlabeled_dataset = np.array([3.6, 4.1, 2.9, 4.8, 3.8, 4.2, 4.7, 2.6, 3.1, 4.4, 3.5, 4.2, 2.8, 4.9, 3.7, 4.1, 4.8, 2.5, 3.2, 4.5])

# Estimate the mean and standard deviation of the labeled dataset
mean_labeled = np.mean(labeled_dataset)
std_labeled = np.std(labeled_dataset)

# Estimate the mean of the unlabeled dataset
mean_unlabeled = np.mean(unlabeled_dataset)

# Estimate the rectifier function
rectifier = lambda x: (x - mean_labeled) / std_labeled

# Compute the rectified fluency scores for the unlabeled dataset
rectified_scores = rectifier(unlabeled_dataset)

# Compute the confidence interval for the rectified fluency scores
confidence_interval = [np.mean(rectified_scores) - 1.96 * np.std(rectified_scores), np.mean(rectified_scores) + 1.96 * np.std(rectified_scores)]

# Print the confidence interval
print(f"The 95% confidence interval for the rectified fluency scores of the unlabeled dataset is [{confidence_interval[0]:.2f}, {confidence_interval[1]:.2f}].")


The 95% confidence interval for the rectified fluency scores of the unlabeled dataset is [-0.75, 1.57].


In [3]:
print(unlabeled_dataset)

[3.6 4.1 2.9 4.8 3.8 4.2 4.7 2.6 3.1 4.4 3.5 4.2 2.8 4.9 3.7 4.1 4.8 2.5
 3.2 4.5]


In [2]:
print(rectified_scores)

[ 0.23643312  0.63048832 -0.31524416  1.18216561  0.3940552   0.70929937
  1.10335457 -0.55167728 -0.15762208  0.86692145  0.15762208  0.70929937
 -0.3940552   1.26097665  0.31524416  0.63048832  1.18216561 -0.63048832
 -0.07881104  0.94573249]


In [1]:
from moverscore_v2 import get_idf_dict, word_mover_score, sentence_score
from collections import defaultdict


moverscore = sentence_score(sys, refs)

ModuleNotFoundError: No module named 'nlpeval'

Hypothetical ratings for Annotator 2: [4, 3, 5, 2, 4, 3, 5, 2, 4, 3]
Hypothetical ratings for Annotator 3: [4, 3, 5, 2, 4, 3, 5, 2, 4, 3]
